In [1]:
import pandas as pd

In [2]:
import transformers
import torch
import random
from tqdm import tqdm

/home/eshaant/cross-task-ICL/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from datasets import load_dataset

In [4]:
import os

In [5]:
import json

# Low Resource

## Hate Explain t1

In [36]:
dataset_test= load_dataset("hatexplain", split='test')
def get_majority_hatexplain_t1(l):
          if l[0]==l[1]:
                    return l[0]
          elif l[1]==l[2]:
                    return l[1]
          else:
                    return l[0]

test_id=[]
test_s=[]
test_l=[]

label2text={
    0:'hate',
    1:'normal',
    2:'offensive'
}
for d in dataset_test:
    test_id.append(d['id'])
    test_l.append([label2text[get_majority_hatexplain_t1(d['annotators']['label'])]])
    test_s.append(' '.join(d['post_tokens']))
    
df={
    'id':test_id,
    'label':test_l,
    'sentence':test_s
}
test_df=pd.DataFrame(df)
test_s_df=test_df.to_dict('records')

Found cached dataset hatexplain (/home/eshaant/.cache/huggingface/datasets/hatexplain/plain_text/1.0.0/df474d8d8667d89ef30649bf66e9c856ad8305bef4bc147e8e31cbdf1b8e0249)


In [37]:
test_s_df[0]

{'id': '1178610029273976833_twitter',
 'label': ['normal'],
 'sentence': '<user> men can not be raped can not be abused that why they call it violence against women and children because men are always the perpetrators women come first even before children'}

In [18]:
data_to_save={
          'dataset_name':'hatexplain_t1',
          'prompt_temp_id':'001',
          'data':test_s_df,
          'set':'test',
          'eval-metric':'acc',
          'exact-match':True

}

import json

with open('data/low-resource/hatexplain_t1_test.json', 'w') as openfile:
 
    # Reading from json file
    json.dump(data_to_save, openfile)
for line in open('data/low-resource/hatexplain_t1_test.json','r'):
    test_set=json.loads(line)

In [1]:
from utils.aligner import Few_Shot_Aligner
import torch
from utils.lm import create_embeddings
from utils.data import combined_source_dataset

few_shot_aligner=Few_Shot_Aligner()

source_data=few_shot_aligner.all_source_tasks

dataset=combined_source_dataset(source_data)

/home/eshaant/cross-task-ICL/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## HateExplain2

In [8]:
dataset_test= load_dataset("hatexplain", split='test')

test_id=[]
test_s=[]
test_l=[]

for d in dataset_test:
    target=set([x for y in d['annotators']['target'] for x in y])

    if 'None' in target:
        target.remove('None')   

    if len(target)==0:
        continue 

    target=list(target)
     
    test_id.append(d['id'])
    test_l.append(target)
    test_s.append(' '.join(d['post_tokens']))    
df={
    'id':test_id,
    'label':test_l,
    'sentence':test_s
}
test_df=pd.DataFrame(df)
test_s_df=test_df.to_dict('records')

Found cached dataset hatexplain (/home/eshaant/.cache/huggingface/datasets/hatexplain/plain_text/1.0.0/df474d8d8667d89ef30649bf66e9c856ad8305bef4bc147e8e31cbdf1b8e0249)


In [41]:
test_s_df[0]

{'id': '1165785686903009283_twitter',
 'label': ['Refugee'],
 'sentence': '<user> you are missing an essential premise whether any of those immigrants committed any crimes dumbass'}

In [42]:
test_s_df

[{'id': '1165785686903009283_twitter',
  'label': ['Refugee'],
  'sentence': '<user> you are missing an essential premise whether any of those immigrants committed any crimes dumbass'},
 {'id': '1252707503441313794_twitter',
  'label': ['Other'],
  'sentence': '<user> <user> why are you repeating yourself are you a little retarded'},
 {'id': '1103385226921762816_twitter',
  'label': ['Jewish', 'Arab', 'Islam'],
  'sentence': '<user> <user> well she ’ muslim so of course she hates the jews jew hatred is one of the defining feature of islam'},
 {'id': '1169443635869487105_twitter',
  'label': ['African'],
  'sentence': '<user> lol not me i don ’ t deal with bms not outside children ’ ss i leave that ghetto mess for everybody else 😂 😂 😂'},
 {'id': '17372469_gab',
  'label': ['Nonreligious', 'Other', 'African', 'Women', 'Minority'],
  'sentence': 'an these nigger biches look like godzilla nasty'},
 {'id': '1168407844095483904_twitter',
  'label': ['Other'],
  'sentence': 'yeah humnh what h

In [40]:
data_to_save={
          'dataset_name':'hatexplain_t2',
          'prompt_temp_id':'001',
          'data':test_s_df,
          'set':'test',
          'eval-metric':'acc',
          'exact-match':True

}

import json

with open('data/low-resource/hatexplain_t2_test.json', 'w') as openfile:
 
    # Reading from json file
    json.dump(data_to_save, openfile)
for line in open('data/low-resource/hatexplain_t2_test.json','r'):
    test_set=json.loads(line)

## Commonsese_qa

In [47]:
dataset[0]

{'id': '1afa02df02c908a558b4036e80242fac',
 'question': 'A revolving door is convenient for two direction travel, but it also serves as a security measure at a what?',
 'question_concept': 'revolving door',
 'choices': {'label': ['A', 'B', 'C', 'D', 'E'],
  'text': ['bank', 'library', 'department store', 'mall', 'new york']},
 'answerKey': 'A'}

In [113]:
dataset= load_dataset('commonsense_qa')['validation']
test_id=[]
test_s=[]
test_l=[]

label2text={
    'A':0,
    'B':1,
    'C':2,
    'D':3,
    'E':4
}

text2label={
    0:'\nA. ',
    1:'\nB. ',
    2:'\nC. ',
    3:'\nD. ',
    4:'\nE. '
}

for d in dataset:
    test_id.append(d['id'])
    test_l.append([d['answerKey']])

    q=d['question']
    for i,a in enumerate(d['choices']['text']):
        q+=' '+text2label[i]+a

    test_s.append(q)

df={
    'id':test_id,
    'label':test_l,
    'sentence':test_s
}

test_df=pd.DataFrame(df)
test_s_df=test_df.to_dict('records')

Found cached dataset commonsense_qa (/home/eshaant/.cache/huggingface/datasets/commonsense_qa/default/1.0.0/28d68f56649a7f0c23bc68eae850af914aa03f95f810011ae8cf58cc5ff5051b)
100%|██████████| 3/3 [00:00<00:00, 450.92it/s]


In [114]:
data_to_save={
          'dataset_name':'commonsense_qa',
          'prompt_temp_id':'002',
          'data':test_s_df,
          'set':'test',
          'eval-metric':'acc',
          'exact-match':True

}

import json

with open('data/low-resource/commonsense_qa_test.json', 'w') as openfile:
 
    # Reading from json file
    json.dump(data_to_save, openfile)
for line in open('data/low-resource/commonsense_qa_test.json','r'):
    test_set=json.loads(line)

In [115]:
test_set['data'][0]

{'id': '1afa02df02c908a558b4036e80242fac',
 'label': ['A'],
 'sentence': 'A revolving door is convenient for two direction travel, but it also serves as a security measure at a what? \nA. bank \nB. library \nC. department store \nD. mall \nE. new york'}

## Medmcqa

In [110]:
dataset= load_dataset('medmcqa')['validation']
test_id=[]
test_s=[]
test_l=[]

text2label={
    0:'A',
    1:'B',
    2:'C',
    3:'D'
}

for d in dataset:
    test_id.append(d['id'])
    test_l.append([text2label[d['cop']]])

    q=d['question']+' \nA. '+d['opa']+' \nB. '+d['opb']+' \nC. '+d['opc']+' \nD. '+d['opd']

    test_s.append(q)

df={
    'id':test_id,
    'label':test_l,
    'sentence':test_s
}
test_df=pd.DataFrame(df)
test_s_df=test_df.to_dict('records')

Found cached dataset medmcqa (/home/eshaant/.cache/huggingface/datasets/medmcqa/default/1.1.0/f2fdfa9ccfbf9d148c0639e6afe3379f3c7e95c4d52d5e68ec1156e5004bd880)
100%|██████████| 3/3 [00:00<00:00, 309.46it/s]


In [111]:
data_to_save={
          'dataset_name':'medmcqa',
          'prompt_temp_id':'002',
          'data':test_s_df,
          'set':'test',
          'eval-metric':'acc',
          'exact-match':True

}

import json

with open('data/low-resource/medmcqa_test.json', 'w') as openfile:
 
    # Reading from json file
    json.dump(data_to_save, openfile)
for line in open('data/low-resource/medmcqa_test.json','r'):
    test_set=json.loads(line)

In [112]:
test_set['data'][0]

{'id': '45258d3d-b974-44dd-a161-c3fccbdadd88',
 'label': ['A'],
 'sentence': 'Which of the following is not true for myelinated nerve fibers: \nA. Impulse through myelinated fibers is slower than non-myelinated fibers \nB. Membrane currents are generated at nodes of Ranvier \nC. Saltatory conduction of impulses is seen \nD. Local anesthesia is effective only when the nerve is not covered by myelin sheath'}

## Social_i_qa

In [107]:
dataset= load_dataset('social_i_qa')['validation']
test_id=[]
test_s=[]
test_l=[]
test_c=[]
id=1

text2label={
    '1':'A',
    '2':'B',
    '3':'C'
}
label2text={
    1:'answerA',
    2:'answerB',
    3:'answerC'
}


for d in dataset:
          test_id.append(id)
          id+=1

          test_l.append([text2label[d['label']]])
          test_c.append(d['context'])

          q=d['question']+' \nA. '+d['answerA']+' \nB. '+d['answerB']+' \nC. '+d['answerC']
          test_s.append(q)

df={
    'id':test_id,
    'label':test_l,
    'sentence':test_s,
    'context':test_c
}
test_df=pd.DataFrame(df)
test_s_df=test_df.to_dict('records')

Found cached dataset social_i_qa (/home/eshaant/.cache/huggingface/datasets/social_i_qa/default/0.1.0/674d85e42ac7430d3dcd4de7007feaffcb1527c535121e09bab2803fbcc925f8)
100%|██████████| 2/2 [00:00<00:00, 442.60it/s]


In [108]:
data_to_save={
          'dataset_name':'social_i_qa',
          'prompt_temp_id':'003',
          'data':test_s_df,
          'set':'test',
          'eval-metric':'acc',
          'exact-match':True

}

import json

with open('data/low-resource/social_i_qa_test.json', 'w') as openfile:
 
    # Reading from json file
    json.dump(data_to_save, openfile)
for line in open('data/low-resource/social_i_qa_test.json','r'):
    test_set=json.loads(line)

In [109]:
test_set

{'dataset_name': 'social_i_qa',
 'prompt_temp_id': '003',
 'data': [{'id': 1,
   'label': ['C'],
   'sentence': 'What does Tracy need to do before this?\nA. make a new plan\nB. Go home and see Riley\nC. Find somewhere to go',
   'context': "Tracy didn't go home that evening and resisted Riley's attacks."},
  {'id': 2,
   'label': ['A'],
   'sentence': 'How would you describe Sydney?\nA. sympathetic\nB. like a person who was unable to help\nC. incredulous',
   'context': 'Sydney walked past a homeless woman asking for change but did not have any money they could give to her. Sydney felt bad afterwards.'},
  {'id': 3,
   'label': ['B'],
   'sentence': 'What will patients want to do next?\nA. write new laws\nB. get petitions signed\nC. live longer',
   'context': "Sasha protected the patients' rights by making new laws regarding cancer drug trials."},
  {'id': 4,
   'label': ['A'],
   'sentence': "How would Jordan feel afterwards?\nA. horrible that he let his friends down on the camping t

## ComVE_t1

In [93]:
df_test=pd.read_csv('data/raw/ComVE/subtaskA_test_data.csv').to_dict('records')
df_test_gl=pd.read_csv('data/raw/ComVE/subtaskA_gold_answers.csv',names=['id','label']).set_index('id')['label'].to_dict()

In [94]:
df_test[0]

{'id': 1175,
 'sent0': 'He loves to stroll at the park with his bed',
 'sent1': 'He loves to stroll at the park with his dog.'}

In [101]:
test_id=[]
test_s1=[]
test_s2=[]
test_l=[]

label2text={
    0:'1',
    1:'2'
}


for d in df_test:
          test_id.append(d['id'])

          test_l.append([label2text[df_test_gl[d['id']]]])
          test_s1.append(d['sent0'])
          test_s2.append(d['sent1'])

df={
    'id':test_id,
    'label':test_l,
    'sentence1':test_s1,
    'sentence2':test_s2
}
test_df=pd.DataFrame(df)
test_s_df=test_df.to_dict('records')

In [102]:
data_to_save={
          'dataset_name':'comve_t1',
          'prompt_temp_id':'004',
          'data':test_s_df,
          'set':'test',
          'eval-metric':'acc',
          'exact-match':True

}

import json

with open('data/low-resource/comve_t1_test.json', 'w') as openfile:
 
    # Reading from json file
    json.dump(data_to_save, openfile)
for line in open('data/low-resource/comve_t1_test.json','r'):
    test_set=json.loads(line)

In [103]:
test_set['data'][1]

{'id': 452,
 'label': ['1'],
 'sentence1': 'The inverter was able to power the continent.',
 'sentence2': 'The inverter was able to power the house'}

## ComVE_t2

In [104]:
df_test=pd.read_csv('data/raw/ComVE/subtaskB_test_data.csv').to_dict('records')
df_test_gl=pd.read_csv('data/raw/ComVE/subtaskB_gold_answers.csv',names=['id','label']).set_index('id')['label'].to_dict()

In [105]:
df_test[0]

{'id': 1175,
 'FalseSent': 'He loves to stroll at the park with his bed',
 'OptionA': 'A bed is too heavy to carry with when strolling at a park',
 'OptionB': 'walking at a park is good for health',
 'OptionC': 'Some beds are big while some are smaller'}

In [116]:
test_id=[]
test_s=[]
test_l=[]


for d in df_test:
          test_id.append(d['id'])

          test_l.append([df_test_gl[d['id']]])
          s=d['FalseSent']+ '\n A. '+d['OptionA']+'\n B. '+d['OptionB']+'\n C. '+d['OptionC']
          test_s.append(s)

df={
    'id':test_id,
    'label':test_l,
    'sentence':test_s
}
test_df=pd.DataFrame(df)
test_s_df=test_df.to_dict('records')

In [117]:
data_to_save={
          'dataset_name':'comve_t2',
          'prompt_temp_id':'001',
          'data':test_s_df,
          'set':'test',
          'eval-metric':'acc',
          'exact-match':True

}

import json

with open('data/low-resource/comve_t2_test.json', 'w') as openfile:
 
    # Reading from json file
    json.dump(data_to_save, openfile)
for line in open('data/low-resource/comve_t2_test.json','r'):
    test_set=json.loads(line)

In [118]:
test_set['data'][0]

{'id': 1175,
 'label': ['A'],
 'sentence': 'He loves to stroll at the park with his bed\n A. A bed is too heavy to carry with when strolling at a park\n B. walking at a park is good for health\n C. Some beds are big while some are smaller'}

## SciQ

In [13]:
dataset= load_dataset('sciq')['test']
test_id=[]
test_s=[]
test_l=[]
id=1
import random
text2label={
    'A':'distractor3',
    'B':'distractor1',
    'C':'distractor2',
    'D':'correct_answer'
}

for d in dataset:

          k=['A','B','C','D']
          k_=['A','B','C','D']
          random.shuffle(k)
          q=d['question']
          
          for i,l in zip(k,k_):
                    op=text2label[i]
                    if op=='correct_answer':
                              test_l.append([l])

                    q+=f'\n{l}. '+d[op]
                    



          test_id.append(id)
          id+=1
          test_s.append(q)

df={
    'id':test_id,
    'label':test_l,
    'sentence':test_s
}
test_df=pd.DataFrame(df)
test_s_df=test_df.to_dict('records')

Found cached dataset sciq (/home/eshaant/.cache/huggingface/datasets/sciq/default/0.1.0/50e5c6e3795b55463819d399ec417bfd4c3c621105e00295ddb5f3633d708493)
100%|██████████| 3/3 [00:00<00:00, 589.81it/s]


In [14]:
data_to_save={
          'dataset_name':'sciq',
          'prompt_temp_id':'002',
          'data':test_s_df,
          'set':'test',
          'eval-metric':'acc',
          'exact-match':True

}

import json

with open('data/low-resource/sciq_test.json', 'w') as openfile:
 
    # Reading from json file
    json.dump(data_to_save, openfile)
for line in open('data/low-resource/sciq_test.json','r'):
    test_set=json.loads(line)

In [15]:
test_set['data']

[{'id': 1,
  'label': ['C'],
  'sentence': 'Compounds that are capable of accepting electrons, such as o 2 or f2, are called what?\nA. residues\nB. antioxidants\nC. oxidants\nD. Oxygen'},
 {'id': 2,
  'label': ['D'],
  'sentence': 'What term in biotechnology means a genetically exact copy of an organism?\nA. male\nB. phenotype\nC. adult\nD. clone'},
 {'id': 3,
  'label': ['A'],
  'sentence': 'Vertebrata are characterized by the presence of what?\nA. backbone\nB. Thumbs\nC. Muscles\nD. Bones'},
 {'id': 4,
  'label': ['D'],
  'sentence': 'What is the height above or below sea level called?\nA. variation\nB. latitude\nC. depth\nD. elevation'},
 {'id': 5,
  'label': ['B'],
  'sentence': 'Ice cores, varves and what else indicate the environmental conditions at the time of their creation?\nA. magma\nB. tree rings\nC. fossils\nD. mountain ranges'},
 {'id': 6,
  'label': ['D'],
  'sentence': 'What chemical signals in plants control different processes?\nA. produce hormones\nB. nitrogen hormone

## CaseHold

In [17]:
dataset= load_dataset('casehold/casehold','all')['validation']
test_id=[]
test_s=[]
test_l=[]
import random
text2label={
    'A':'holding_0',
    'B':'holding_1',
    'C':'holding_2',
    'D':'holding_3',
    'E':'holding_4'
}

label2text={
    '0':'A',
    '1':'B',
    '2':'C',
    '3':'D',
    '4':'E'
}

for d in dataset:
    k_=['A','B','C','D','E']
    random.shuffle(k)
    q=d['citing_prompt']
          
    for l in k_:
        q+=f'\n{l}. '+d[text2label[l]]
                    



    test_id.append(d['example_id'])
    test_s.append(q)
    test_l.append([label2text[d['label']]])

df={
    'id':test_id,
    'label':test_l,
    'sentence':test_s
}
test_df=pd.DataFrame(df)
test_s_df=test_df.to_dict('records')

Found cached dataset casehold (/home/eshaant/.cache/huggingface/datasets/casehold___casehold/all/1.1.0/56b9c04b77aae7ed78ede73bbabef236b72e73e2bbda22f66dfcdb8465985b0b)
100%|██████████| 3/3 [00:00<00:00, 452.85it/s]


In [18]:
data_to_save={
          'dataset_name':'casehold',
          'prompt_temp_id':'001',
          'data':test_s_df,
          'set':'test',
          'eval-metric':'acc',
          'exact-match':True

}

import json

with open('data/low-resource/casehold_test.json', 'w') as openfile:
 
    # Reading from json file
    json.dump(data_to_save, openfile)
for line in open('data/low-resource/casehold_test.json','r'):
    test_set=json.loads(line)

In [19]:
test_set['data'][0]

{'id': 42508,
 'label': ['D'],
 'sentence': 'was acting to obtain a benefit on behalf of a charitable ... organization.” U.S.S.G. § 2B1.1 cmt. 8(B). As the district court saw it and as the government sees it, Webster deserves the enhancement. He pretended to “act[ ] on behalf of a charitable ... organization,” U.S.S.G. § 2Bl.l(b)(9)(A), when he solicited personal information from the victims on behalf of fake charities. As Webster sees it, the enhancement does not apply. In his view, the commentary limits the application of the charity enhancement, and he was not acting to obtain a benefit on behalf of a charitable organization (as the commentary seems to require). As a general matter, the text of a guideline trumps commentary about it. See Stinson v. United States, 508 U.S. 36, 38, 113 S.Ct. 1913, 123 L.Ed.2d 598 (1993) (<HOLDING>). But we need not resolve whether the\nA. recognizing the guidelines commentary is authoritative\nB. holding that a sentencing guideline prevails over its c

# High Resource

ADD LABEL NUMBER

In [6]:
k=100

## Glue

### sst2

In [7]:
dataset= load_dataset('glue','sst2')['train']
test_id=[]
test_s=[]
test_l=[]
import random
text2label={
    1:'positive',
    0:'negative'
}

for d in dataset:
          test_id.append(d['idx'])
          test_s.append(d['sentence'])
          test_l.append(text2label[d['label']])

df={
    'id':test_id,
    'label':test_l,
    'sentence':test_s
}
test_df=pd.DataFrame(df)
test_df=test_df.sample(k)
test_s_df=test_df.to_dict('records')
print(len(test_s_df))

Found cached dataset glue (/home/eshaant/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|██████████| 3/3 [00:00<00:00, 14.59it/s]


100


In [8]:
data_to_save={
          'dataset_name':'sst2',
          'prompt_temp_id':'001',
          'data':test_s_df,
          'set':'train'

}

import json

with open('data/high-resource/sst2_test.json', 'w') as openfile:
 
    # Reading from json file
    json.dump(data_to_save, openfile)
for line in open('data/high-resource/sst2_test.json','r'):
    test_set=json.loads(line)

In [9]:
test_set['data'][0]

{'id': 5812, 'label': 'negative', 'sentence': 'a gory slash-fest '}

### WNLI

In [10]:
dataset= load_dataset('glue','wnli')['train']
test_id=[]
test_s1=[]
test_s2=[]
test_l=[]
import random
text2label={
    1:'entailment',
    0:'not entailment'
}

for d in dataset:
          test_id.append(d['idx'])
          test_s1.append(d['sentence1'])
          test_s2.append(d['sentence2'])
          test_l.append(text2label[d['label']])

df={
    'id':test_id,
    'label':test_l,
    'sentence1':test_s1,
    'sentence2':test_s2
}
test_df=pd.DataFrame(df)
test_df=test_df.sample(k)
test_s_df=test_df.to_dict('records')
print(len(test_s_df))

Found cached dataset glue (/home/eshaant/.cache/huggingface/datasets/glue/wnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|██████████| 3/3 [00:00<00:00, 113.95it/s]

100


In [11]:
data_to_save={
          'dataset_name':'wnli',
          'prompt_temp_id':'004',
          'data':test_s_df,
          'set':'train'

}

import json

with open('data/high-resource/wnli_test.json', 'w') as openfile:
 
    # Reading from json file
    json.dump(data_to_save, openfile)
for line in open('data/high-resource/wnli_test.json','r'):
    test_set=json.loads(line)
test_set['data'][0]

{'id': 515,
 'label': 'not entailment',
 'sentence1': 'Bill passed the half-empty plate to John because he was hungry.',
 'sentence2': 'Bill was hungry.'}

### RTE

In [12]:
dataset_name='rte'
dataset= load_dataset('glue',dataset_name)['train']
test_id=[]
test_s1=[]
test_s2=[]
test_l=[]
import random
text2label={
    1:'entailment',
    0:'not entailment'
}

for d in dataset:
          test_id.append(d['idx'])
          test_s1.append(d['sentence1'])
          test_s2.append(d['sentence2'])
          test_l.append(text2label[d['label']])

df={
    'id':test_id,
    'label':test_l,
    'sentence1':test_s1,
    'sentence2':test_s2
}
test_df=pd.DataFrame(df)
test_df=test_df.sample(k)
test_s_df=test_df.to_dict('records')
print(len(test_s_df))

Found cached dataset glue (/home/eshaant/.cache/huggingface/datasets/glue/rte/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|██████████| 3/3 [00:00<00:00, 42.49it/s]


100


In [13]:

data_to_save={
          'dataset_name':dataset_name,
          'prompt_temp_id':'004',
          'data':test_s_df,
          'set':'train'

}

import json

with open(f'data/high-resource/{dataset_name}_test.json', 'w') as openfile:
 
    # Reading from json file
    json.dump(data_to_save, openfile)
for line in open(f'data/high-resource/{dataset_name}_test.json','r'):
    test_set=json.loads(line)
test_set['data'][0]

{'id': 2145,
 'label': 'not entailment',
 'sentence1': 'New York Times reporter Judith Miller broke her media silence by writing a column the newspaper published on Sunday. Her lengthy story recounted two testimonies, September 30 and October 12, she gave to a U.S. federal grand jury seeking information on the source of the Valerie Plame leak. The column described the testimony as one that relied heavily on her reporting notes given to special prosecutor Patrick Fitzgerald between her first and second  appearances.',
 'sentence2': 'Judith Miller works for the New York Times.'}

### qqp

In [14]:
dataset_name='qqp'
dataset= load_dataset('glue',dataset_name)['train']
test_id=[]
test_s1=[]
test_s2=[]
test_l=[]
import random
text2label={
    1:'duplicate',
    0:'not duplicate'
}

for d in dataset:
          test_id.append(d['idx'])
          test_s1.append(d['question1'])
          test_s2.append(d['question2'])
          test_l.append(text2label[d['label']])

df={
    'id':test_id,
    'label':test_l,
    'sentence1':test_s1,
    'sentence2':test_s2
}
test_df=pd.DataFrame(df)
test_df=test_df.sample(k)
test_s_df=test_df.to_dict('records')
print(len(test_s_df))

Found cached dataset glue (/home/eshaant/.cache/huggingface/datasets/glue/qqp/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|██████████| 3/3 [00:00<00:00, 10.54it/s]


100


In [15]:

data_to_save={
          'dataset_name':dataset_name,
          'prompt_temp_id':'005',
          'data':test_s_df,
          'set':'train'

}

import json

with open(f'data/high-resource/{dataset_name}_test.json', 'w') as openfile:
 
    # Reading from json file
    json.dump(data_to_save, openfile)
for line in open(f'data/high-resource/{dataset_name}_test.json','r'):
    test_set=json.loads(line)
test_set['data'][0]

{'id': 101849,
 'label': 'not duplicate',
 'sentence1': 'Is it acceptable to use a fake Facebook account to join Quora?',
 'sentence2': 'Has Quora become too hard to use?'}

### qnli

In [16]:
dataset_name='qnli'
dataset= load_dataset('glue',dataset_name)['train']
test_id=[]
test_s1=[]
test_s2=[]
test_l=[]
import random
text2label={
    1:'entailment',
    0:'not entailment'
}

for d in dataset:
          test_id.append(d['idx'])
          test_s1.append(d['question'])
          test_s2.append(d['sentence'])
          test_l.append(text2label[d['label']])

df={
    'id':test_id,
    'label':test_l,
    'sentence1':test_s1,
    'sentence2':test_s2
}
test_df=pd.DataFrame(df)
test_df=test_df.sample(k)
test_s_df=test_df.to_dict('records')
print(len(test_s_df))

Found cached dataset glue (/home/eshaant/.cache/huggingface/datasets/glue/qnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|██████████| 3/3 [00:00<00:00, 24.42it/s]


100


In [17]:

data_to_save={
          'dataset_name':dataset_name,
          'prompt_temp_id':'006',
          'data':test_s_df,
          'set':'train'

}

import json

with open(f'data/high-resource/{dataset_name}_test.json', 'w') as openfile:
 
    # Reading from json file
    json.dump(data_to_save, openfile)
for line in open(f'data/high-resource/{dataset_name}_test.json','r'):
    test_set=json.loads(line)
test_set['data'][0]

{'id': 37803,
 'label': 'entailment',
 'sentence1': '1000 men occupied which cities?',
 'sentence2': 'Cajen attained several advantages over the liberals within the state, but soon lost his standing due to a strong resurgence of the liberal forces within the state.'}

### MRPC

In [18]:
dataset_name='mrpc'
dataset= load_dataset('glue',dataset_name)['train']
test_id=[]
test_s1=[]
test_s2=[]
test_l=[]
import random
text2label={
    1:'equivalent',
    0:'not equivalent'
}

for d in dataset:
          test_id.append(d['idx'])
          test_s1.append(d['sentence1'])
          test_s2.append(d['sentence2'])
          test_l.append(text2label[d['label']])

df={
    'id':test_id,
    'label':test_l,
    'sentence1':test_s1,
    'sentence2':test_s2
}
test_df=pd.DataFrame(df)
test_df=test_df.sample(k)
test_s_df=test_df.to_dict('records')
print(len(test_s_df))

Found cached dataset glue (/home/eshaant/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|██████████| 3/3 [00:00<00:00, 71.22it/s]


100


In [19]:

data_to_save={
          'dataset_name':dataset_name,
          'prompt_temp_id':'004',
          'data':test_s_df,
          'set':'train'

}

import json

with open(f'data/high-resource/{dataset_name}_test.json', 'w') as openfile:
 
    # Reading from json file
    json.dump(data_to_save, openfile)
for line in open(f'data/high-resource/{dataset_name}_test.json','r'):
    test_set=json.loads(line)
test_set['data'][0]

{'id': 1339,
 'label': 'not equivalent',
 'sentence1': "A representative for Phoenix-based U-Haul declined to comment on the case before the judge 's opinion was released .",
 'sentence2': 'Anthony Citrano , a representative for WhenU , declined to comment on the case .'}

### Mnli

In [20]:
dataset_name='mnli'
dataset= load_dataset('glue',dataset_name)['train']
test_id=[]
test_s1=[]
test_s2=[]
test_l=[]
import random
text2label={
    2:'contradiction',
    1:'neutral',
    0:'entailment'
}

for d in dataset:
          test_id.append(d['idx'])
          test_s1.append(d['premise'])
          test_s2.append(d['hypothesis'])
          test_l.append(text2label[d['label']])

df={
    'id':test_id,
    'label':test_l,
    'sentence1':test_s1,
    'sentence2':test_s2
}
test_df=pd.DataFrame(df)
test_df=test_df.sample(k)
test_s_df=test_df.to_dict('records')
print(len(test_s_df))

Found cached dataset glue (/home/eshaant/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|██████████| 5/5 [00:00<00:00,  8.86it/s]


100


In [21]:
data_to_save={
          'dataset_name':dataset_name,
          'prompt_temp_id':'007',
          'data':test_s_df,
          'set':'train'

}

import json

with open(f'data/high-resource/{dataset_name}_test.json', 'w') as openfile:
 
    # Reading from json file
    json.dump(data_to_save, openfile)
for line in open(f'data/high-resource/{dataset_name}_test.json','r'):
    test_set=json.loads(line)
test_set['data'][0]

{'id': 234309,
 'label': 'neutral',
 'sentence1': 'This chapter prescribes reporting standards and provides guidance to auditors reporting on performance audits in accordance with generally accepted government auditing standards (GAGAS).',
 'sentence2': 'Auditors benefit from auditing standard practices.'}

### Cola

In [22]:
dataset_name='cola'
dataset= load_dataset('glue',dataset_name)['train']
test_id=[]
test_s=[]
test_l=[]
import random
text2label={
    1:'acceptable',
    0:'unacceptable'
}

for d in dataset:
          test_id.append(d['idx'])
          test_s.append(d['sentence'])
          test_l.append(text2label[d['label']])

print(len(test_id))
print(len(test_s))
print(len(test_l))
df={
    'id':test_id,
    'label':test_l,
    'sentence':test_s
}
test_df=pd.DataFrame(df)
test_df=test_df.sample(k)
test_s_df=test_df.to_dict('records')
print(len(test_s_df))

Found cached dataset glue (/home/eshaant/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
100%|██████████| 3/3 [00:00<00:00, 621.65it/s]


8551
8551
8551
100


In [23]:
data_to_save={
          'dataset_name':dataset_name,
          'prompt_temp_id':'001',
          'data':test_s_df,
          'set':'train'

}

import json

with open(f'data/high-resource/{dataset_name}_test.json', 'w') as openfile:
 
    # Reading from json file
    json.dump(data_to_save, openfile)
for line in open(f'data/high-resource/{dataset_name}_test.json','r'):
    test_set=json.loads(line)
test_set['data'][0]

{'id': 2727,
 'label': 'unacceptable',
 'sentence': 'We offered a job behind her.'}

## Web QA

In [24]:
dataset_name='web_questions'
dataset= load_dataset(dataset_name)['train']
test_id=[]
test_s=[]
test_l=[]
id=1
import random

for d in dataset:
          test_id.append(id)
          id+=1
          test_s.append(d['question'])
          test_l.append(d['answers'][0])

df={
    'id':test_id,
    'label':test_l,
    'sentence':test_s
}
test_df=pd.DataFrame(df)
test_df=test_df.sample(k)
test_s_df=test_df.to_dict('records')
print(len(test_s_df))

Found cached dataset web_questions (/home/eshaant/.cache/huggingface/datasets/web_questions/default/1.0.0/e6742cc64f6652db0c52cb07b5414e3c001512bf5bde7aa5587353c31db1ed8b)
100%|██████████| 2/2 [00:00<00:00, 461.62it/s]

100


In [25]:
data_to_save={
          'dataset_name':dataset_name,
          'prompt_temp_id':'002',
          'data':test_s_df,
          'set':'train'

}

import json

with open(f'data/high-resource/{dataset_name}_test.json', 'w') as openfile:
 
    # Reading from json file
    json.dump(data_to_save, openfile)
for line in open(f'data/high-resource/{dataset_name}_test.json','r'):
    test_set=json.loads(line)
test_set['data'][0]

{'id': 3767,
 'label': 'Peruvian nuevo sol',
 'sentence': 'what is the money called in peru?'}

## Super_glue

### Boolq

In [26]:
dataset_name='boolq'
dataset= load_dataset(dataset_name)['train']
test_id=[]
test_s=[]
test_c=[]
test_l=[]
id=1
import random


for d in dataset:
          test_id.append(id)
          id+=1
          test_s.append(d['question'])
          test_c.append(d['passage'])
          test_l.append(d['answer'])

df={
    'id':test_id,
    'label':test_l,
    'sentence':test_s,
    'context':test_c
}
test_df=pd.DataFrame(df)
test_df=test_df.sample(k)
test_s_df=test_df.to_dict('records')
print(len(test_s_df))

Found cached dataset boolq (/home/eshaant/.cache/huggingface/datasets/boolq/default/0.1.0/bf0dd57da941c50de94ae3ce3cef7fea48c08f337a4b7aac484e9dddc5aa24e5)
100%|██████████| 2/2 [00:00<00:00, 52.12it/s]


100


In [27]:
data_to_save={
          'dataset_name':dataset_name,
          'prompt_temp_id':'003',
          'data':test_s_df,
          'set':'train'

}

import json

with open(f'data/high-resource/{dataset_name}_test.json', 'w') as openfile:
 
    # Reading from json file
    json.dump(data_to_save, openfile)
for line in open(f'data/high-resource/{dataset_name}_test.json','r'):
    test_set=json.loads(line)
test_set['data'][0]

{'id': 7045,
 'label': False,
 'sentence': 'is your maid of honor supposed to be married',
 'context': 'The principal bridesmaid, if one is so designated, may be called the chief bridesmaid or maid of honor if she is unmarried, or the matron of honor if she is married. A junior bridesmaid is a girl who is clearly too young to be married, but who is included as an honorary bridesmaid. In the United States, typically only the maid/matron of honor and the best man are the official witnesses for the wedding license.'}

## Financial_phrasebank

In [29]:
dataset_name='financial_phrasebank'
dataset= load_dataset(dataset_name,'sentences_allagree')['train']
test_id=[]
test_s=[]
test_l=[]
id=1
import random
text2label={
    1:'neutal',
    2:'positive',
    0:'negative'
}

for d in dataset:
          test_id.append(id)
          id+=1
          test_s.append(d['sentence'])
          test_l.append(text2label[d['label']])

df={
    'id':test_id,
    'label':test_l,
    'sentence':test_s
}
test_df=pd.DataFrame(df)
test_df=test_df.sample(k)
test_s_df=test_df.to_dict('records')
print(len(test_s_df))

Found cached dataset financial_phrasebank (/home/eshaant/.cache/huggingface/datasets/financial_phrasebank/sentences_allagree/1.0.0/550bde12e6c30e2674da973a55f57edde5181d53f5a5a34c1531c53f93b7e141)
100%|██████████| 1/1 [00:00<00:00, 49.12it/s]

100


In [30]:
data_to_save={
          'dataset_name':dataset_name,
          'prompt_temp_id':'001',
          'data':test_s_df,
          'set':'train'

}

import json

with open(f'data/high-resource/{dataset_name}_test.json', 'w') as openfile:
 
    # Reading from json file
    json.dump(data_to_save, openfile)
for line in open(f'data/high-resource/{dataset_name}_test.json','r'):
    test_set=json.loads(line)
test_set['data'][0]

{'id': 890, 'label': 'neutal', 'sentence': 'The forecast for 2012 is 3.3 % .'}

## trec

In [31]:
dataset_name='trec'
dataset= load_dataset(dataset_name)['train']
test_id=[]
test_s=[]
test_l=[]
id=1
import random
text2label={
    0:'abbreviation',
    1:'entity',
    2:'description',
    3:'human being',
    4:'location',
    5:'numeric Value'
}

for d in dataset:
          test_id.append(id)
          id+=1
          test_s.append(d['text'])
          test_l.append(text2label[d['coarse_label']])

df={
    'id':test_id,
    'label':test_l,
    'sentence':test_s
}
test_df=pd.DataFrame(df)
test_df=test_df.sample(k)
test_s_df=test_df.to_dict('records')
print(len(test_s_df))

Found cached dataset trec (/home/eshaant/.cache/huggingface/datasets/trec/default/2.0.0/f2469cab1b5fceec7249fda55360dfdbd92a7a5b545e91ea0f78ad108ffac1c2)
100%|██████████| 2/2 [00:00<00:00, 483.47it/s]


100


In [32]:
data_to_save={
          'dataset_name':dataset_name,
          'prompt_temp_id':'001',
          'data':test_s_df,
          'set':'train',
          'label':6

}

import json

with open(f'data/high-resource/{dataset_name}_test.json', 'w') as openfile:
 
    # Reading from json file
    json.dump(data_to_save, openfile)
for line in open(f'data/high-resource/{dataset_name}_test.json','r'):
    test_set=json.loads(line)
test_set['data'][0]

{'id': 308,
 'label': 'abbreviation',
 'sentence': 'What does NASDAQ stand for ?'}

## Squad

In [33]:
dataset_name='squad'
dataset= load_dataset(dataset_name)['train']
test_id=[]
test_s=[]
test_c=[]
test_l=[]
id=1
import random

for d in dataset:
          test_id.append(d['id'])
          id+=1
          test_s.append(d['question'])
          test_c.append(d['context'])
          test_l.append(d['answers']['text'][0])

df={
    'id':test_id,
    'label':test_l,
    'sentence':test_s,
    'context':test_c
}
test_df=pd.DataFrame(df)
test_df=test_df.sample(k)
test_s_df=test_df.to_dict('records')
print(len(test_s_df))

Found cached dataset squad (/home/eshaant/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)
100%|██████████| 2/2 [00:00<00:00,  7.95it/s]


100


In [34]:
data_to_save={
          'dataset_name':dataset_name,
          'prompt_temp_id':'003',
          'data':test_s_df,
          'set':'train',
          'labels':None

}

import json

with open(f'data/high-resource/{dataset_name}_test.json', 'w') as openfile:
 
    # Reading from json file
    json.dump(data_to_save, openfile)
for line in open(f'data/high-resource/{dataset_name}_test.json','r'):
    test_set=json.loads(line)
test_set['data'][0]

{'id': '572812723acd2414000df3c5',
 'label': 'the Ministry of Defence Police',
 'sentence': 'Which police force in London does not typically engage in police activity with the general public?',
 'context': "Policing in Greater London, with the exception of the City of London, is provided by the Metropolitan Police Service, overseen by the Mayor through the Mayor's Office for Policing and Crime (MOPAC). The City of London has its own police force – the City of London Police. The British Transport Police are responsible for police services on National Rail, London Underground, Docklands Light Railway and Tramlink services. A fourth police force in London, the Ministry of Defence Police, do not generally become involved with policing the general public."}

## ARC-Easy

In [35]:
dataset_name='ARC-Easy'
dataset= load_dataset('ai2_arc',dataset_name)['train']
test_id=[]
test_s=[]
test_l=[]
id=1
import random

text2label={
    'A':'A',
    'B':'B',
    "C":"C",
    "D":"D",
    '2':'A',
    '1':'B',
    "3":"C",
    "4":"D",
}

for d in dataset:
    if len(d['choices']['text'])!=4:
        continue
    test_id.append(d['id'])
    id+=1
    test_s.append(d['question']+'\nA. '+d['choices']['text'][0]+'\nB. '+d['choices']['text'][1]+'\nC. '+d['choices']['text'][2]+'\nD. '+d['choices']['text'][3])
    test_l.append(text2label[d['answerKey']])

df={
    'id':test_id,
    'label':test_l,
    'sentence':test_s
}
test_df=pd.DataFrame(df)
test_df=test_df.sample(k)
test_s_df=test_df.to_dict('records')
print(len(test_s_df))

Found cached dataset ai2_arc (/home/eshaant/.cache/huggingface/datasets/ai2_arc/ARC-Easy/1.0.0/1569c2591ea2683779581d9fb467203d9aa95543bb9b75dcfde5da92529fd7f6)
100%|██████████| 3/3 [00:00<00:00, 90.77it/s]


100


In [36]:
data_to_save={
          'dataset_name':dataset_name,
          'prompt_temp_id':'002',
          'data':test_s_df,
          'set':'train',
          'labels':4

}

import json

with open(f'data/high-resource/{dataset_name}_test.json', 'w') as openfile:
 
    # Reading from json file
    json.dump(data_to_save, openfile)
for line in open(f'data/high-resource/{dataset_name}_test.json','r'):
    test_set=json.loads(line)
test_set['data'][0]

{'id': 'MDSA_2007_5_23',
 'label': 'C',
 'sentence': 'In 2004, Maryland recycled more than 35% of all of the waste the state produced. Which of these benefits does recycling provide to Maryland?\nA. fewer wild animals\nB. decreased farm crops\nC. less trash sent to landfills\nD. decreased use of electricity'}

## conll_ner

In [ ]:
dataset_name='conll2003'
dataset= load_dataset(dataset_name)['train']
test_id=[]
test_s=[]
test_l=[]
id=1
import random

text2label={0:'O', 
1:'B-PER', 
2:'I-PER',
3:'B-ORG',
4:'I-ORG',
5:'B-LOC',
6:'I-LOC',
7:'B-MISC',
8:'I-MISC'
}

for d in dataset:

    l=[]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
    test_id.append(d['id'])
    for i in d['ner_tags']:
        l.append(text2label[i])
    test_s.append(' '.join(d['tokens']))
    test_l.append(' '.join(l))

df={
    'id':test_id,
    'label':test_l,
    'sentence':test_s
}
test_df=pd.DataFrame(df)
test_df=test_df.sample(k)
test_s_df=test_df.to_dict('records')
print(len(test_s_df))

Found cached dataset conll2003 (/home/eshaant/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98)
100%|██████████| 3/3 [00:00<00:00, 218.85it/s]


100


In [ ]:
data_to_save={
          'dataset_name':dataset_name+'_ner',
          'prompt_temp_id':'001',
          'data':test_s_df,
          'set':'train',
          'labels':4

}

import json

with open(f'data/high-resource/{dataset_name}_ner_test.json', 'w') as openfile:
 
    # Reading from json file
    json.dump(data_to_save, openfile)
for line in open(f'data/high-resource/{dataset_name}_ner_test.json','r'):
    test_set=json.loads(line)
test_set['data'][0]

{'id': '13593',
 'label': 'O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O',
 'sentence': 'Police said the thieves intercepted a woman arriving to work at the canteen , forced her to open the safe where takings were kept and made off with the cash .'}

## conll_chunk

In [ ]:
dataset_name='conll2003'
dataset= load_dataset(dataset_name)['train']
test_id=[]
test_s=[]
test_l=[]
id=1
import random

label={'O': 0, 'B-ADJP': 1, 'I-ADJP': 2, 'B-ADVP': 3, 'I-ADVP': 4, 'B-CONJP': 5, 'I-CONJP': 6, 'B-INTJ': 7, 'I-INTJ': 8,
 'B-LST': 9, 'I-LST': 10, 'B-NP': 11, 'I-NP': 12, 'B-PP': 13, 'I-PP': 14, 'B-PRT': 15, 'I-PRT': 16, 'B-SBAR': 17,
 'I-SBAR': 18, 'B-UCP': 19, 'I-UCP': 20, 'B-VP': 21, 'I-VP': 22}

text2label=dict([(v,k) for k,v in label.items()])


for d in dataset:

    l=[]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
    test_id.append(d['id'])
    for i in d['chunk_tags']:
        l.append(text2label[i])
    test_s.append(' '.join(d['tokens']))
    test_l.append(' '.join(l))

df={
    'id':test_id,
    'label':test_l,
    'sentence':test_s
}
test_df=pd.DataFrame(df)
test_df=test_df.sample(k)
test_s_df=test_df.to_dict('records')
print(len(test_s_df))

Found cached dataset conll2003 (/home/eshaant/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98)
100%|██████████| 3/3 [00:00<00:00, 510.19it/s]


100


In [ ]:
data_to_save={
          'dataset_name':dataset_name+'_chunk',
          'prompt_temp_id':'001',
          'data':test_s_df,
          'set':'train',
          'labels':4

}

import json

with open(f'data/high-resource/{dataset_name}_chunk_test.json', 'w') as openfile:
 
    # Reading from json file
    json.dump(data_to_save, openfile)
for line in open(f'data/high-resource/{dataset_name}_chunk_test.json','r'):
    test_set=json.loads(line)
test_set['data'][0]

{'id': '2859',
 'label': 'B-NP I-NP B-VP B-NP B-VP B-ADJP O O B-NP B-PP B-NP I-NP B-VP I-VP B-ADVP B-VP B-NP B-ADVP B-NP B-ADVP B-VP B-NP I-NP O B-VP B-NP I-NP B-VP I-VP B-PRT B-NP I-NP O',
 'sentence': 'The detective said details were sketchy , but two of the teenagers were reportedly downstairs watching television when White allegedly entered the house and told the girls to take off their clothes .'}

## conll_POS

In [ ]:
dataset_name='conll2003'
dataset= load_dataset(dataset_name)['train']
test_id=[]
test_s=[]
test_l=[]
id=1
import random

label={'"': 0, "''": 1, '#': 2, '$': 3, '(': 4, ')': 5, ',': 6, '.': 7, ':': 8, '``': 9, 'CC': 10, 'CD': 11, 'DT': 12,
 'EX': 13, 'FW': 14, 'IN': 15, 'JJ': 16, 'JJR': 17, 'JJS': 18, 'LS': 19, 'MD': 20, 'NN': 21, 'NNP': 22, 'NNPS': 23,
 'NNS': 24, 'NN|SYM': 25, 'PDT': 26, 'POS': 27, 'PRP': 28, 'PRP$': 29, 'RB': 30, 'RBR': 31, 'RBS': 32, 'RP': 33,
 'SYM': 34, 'TO': 35, 'UH': 36, 'VB': 37, 'VBD': 38, 'VBG': 39, 'VBN': 40, 'VBP': 41, 'VBZ': 42, 'WDT': 43,
 'WP': 44, 'WP$': 45, 'WRB': 46}


text2label=dict([(v,k) for k,v in label.items()])


for d in dataset:

    l=[]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
    test_id.append(d['id'])
    for i in d['pos_tags']:
        l.append(text2label[i])
    test_s.append(' '.join(d['tokens']))
    test_l.append(' '.join(l))

df={
    'id':test_id,
    'label':test_l,
    'sentence':test_s
}
test_df=pd.DataFrame(df)
test_df=test_df.sample(k)
test_s_df=test_df.to_dict('records')
print(len(test_s_df))

Found cached dataset conll2003 (/home/eshaant/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98)
100%|██████████| 3/3 [00:00<00:00, 438.78it/s]


100


In [ ]:
data_to_save={
          'dataset_name':dataset_name+'_pos',
          'prompt_temp_id':'001',
          'data':test_s_df,
          'set':'train',
          'labels':4

}

import json

with open(f'data/high-resource/{dataset_name}_pos_test.json', 'w') as openfile:
 
    # Reading from json file
    json.dump(data_to_save, openfile)
for line in open(f'data/high-resource/{dataset_name}_pos_test.json','r'):
    test_set=json.loads(line)
test_set['data'][0]

{'id': '2431', 'label': 'NNS POS CD NNS', 'sentence': "Women 's 200 metres"}

## AG_news

In [ ]:
dataset_name='ag_news'
dataset= load_dataset(dataset_name)['train']
test_id=[]
test_s=[]
test_l=[]
id=1
import random

text2label={
          0:'world',
          1:'sports',
          2:'business',
          3:'technology'
}


for d in dataset:

    test_id.append(id)
    id+=1
    test_s.append(d['text'])
    test_l.append(text2label[d['label']])

df={
    'id':test_id,
    'label':test_l,
    'sentence':test_s
}
test_df=pd.DataFrame(df)
test_df=test_df.sample(k)
test_s_df=test_df.to_dict('records')
print(len(test_s_df))

Found cached dataset ag_news (/home/eshaant/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)
100%|██████████| 2/2 [00:00<00:00, 395.20it/s]


100


In [ ]:
data_to_save={
          'dataset_name':dataset_name,
          'prompt_temp_id':'001',
          'data':test_s_df,
          'set':'train',
          'labels':4

}

import json

with open(f'data/high-resource/{dataset_name}_test.json', 'w') as openfile:
 
    # Reading from json file
    json.dump(data_to_save, openfile)
for line in open(f'data/high-resource/{dataset_name}_test.json','r'):
    test_set=json.loads(line)
test_set['data'][0]

{'id': 50256,
 'label': 'business',
 'sentence': 'Vandals hit Detroit #39;s Osborn High Students came back to Detroit #39;s Osborn High School on Monday morning to find vandals and thieves damaged the building extensively during the weekend.'}

# RL traning data

In [ ]:
k=100

dataset_name='boolq'
dataset= load_dataset(dataset_name)['train']
test_id=[]
test_s=[]
test_c=[]
test_l=[]
id=1
import random


for d in dataset:
          test_id.append(id)
          id+=1
          test_s.append(d['question'])
          test_c.append(d['passage'])
          test_l.append(d['answer'])

df={
    'id':test_id,
    'label':test_l,
    'sentence':test_s,
    'context':test_c
}
test_df=pd.DataFrame(df)
test_df=test_df.sample(k)
test_s_df=test_df.to_dict('records')
print(len(test_s_df))

data_to_save={
          'dataset_name':dataset_name,
          'prompt_temp_id':'003',
          'data':test_s_df,
          'set':'train'

}

import json

with open(f'data/high-resource/{dataset_name}_test.json', 'w') as openfile:
 
    # Reading from json file
    json.dump(data_to_save, openfile)
for line in open(f'data/high-resource/{dataset_name}_test.json','r'):
    test_set=json.loads(line)
test_set['data'][0]